In [ ]:
!pip install PyDrive

In [84]:
import mykeys

mykeys.initGdriveKeyFile(True)

In [1]:
import mykeys

mykeys.setOsEnv()

아래 링크를 복사하여 웹 브라우저에 붙여넣으세요.
https://accounts.google.com/o/oauth2/auth?client_id=35726703810-4v13dfqmilhgv6shlc3cv9i3ktuh73j1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code
Authentication successful.
mykey => set HUGGINGFACEHUB_API_TOKEN is done
mykey => set OPENAI_API_KEY is done
mykey => set LANGCHAIN_API_KEY is done
mykey => set LANGCHAIN_HUB_API_KEY is done
mykey => set GOOGLE_API_KEY is done
mykey => set GOOGLE_CSE_ID is done
mykey => set UPSTAGE_API_KEY is done
mykey => set COHERE_API_KEY is done
mykey => set JINA_API_KEY is done
mykey => set ANTHROPIC_API_KEY is done
mykey => set DEEPL_API_KEY is done
mykey => set TAVILY_API_KEY is done
mykey => set TOGETHER_API_KEY is done


In [4]:
!pip install opencv-contrib-python
!pip install google-cloud-vision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 31.1 MB/s eta 0:00:0031m31.2 MB/s eta 0:00:01
  Using cached grpcio_status-1.74.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [google-cloud-vision]


In [6]:
import os
import io
import numpy as np
import platform
from PIL import ImageFont, ImageDraw, Image
 
import cv2
from google.cloud import vision

In [19]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 26.0 MB/s eta 0:00:00 MB/s eta 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [matplotlib] 4/5 [matplotlib]


In [20]:
import matplotlib as plt

def plt_imshow(title='image', img=None, figsize=(8 ,5)):
    plt.figure(figsize=figsize)
 
    if type(img) == list:
        if type(title) == list:
            titles = title
        else:
            titles = []
 
            for i in range(len(img)):
                titles.append(title)
 
        for i in range(len(img)):
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
 
            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])
 
        plt.show()
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 
        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()

In [8]:
def putText(image, text, x, y, color=(0, 255, 0), font_size=22):
    if type(image) == np.ndarray:
        color_coverted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(color_coverted)
 
    if platform.system() == 'Darwin':
        font = 'AppleGothic.ttf'
    elif platform.system() == 'Windows':
        font = 'malgun.ttf'
    else:
        font = 'NanumGothic.ttf'
        
    image_font = ImageFont.truetype(font, font_size)
    font = ImageFont.load_default()
    draw = ImageDraw.Draw(image)
 
    draw.text((x, y), text, font=image_font, fill=color)
    
    numpy_image = np.array(image)
    opencv_image = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)
 
    return opencv_image

In [10]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '20250908_GCP_Service_Credentials.json'

client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

In [11]:
path = 'data_ocr/김광무_140.jpg' 
with io.open(path, 'rb') as image_file:
    content = image_file.read()

In [12]:
image = vision.Image(content=content)
 
response = client.text_detection(image=image)
texts = response.text_annotations

In [13]:
img = cv2.imread(path)
roi_img = img.copy()
    
for text in texts:
    print('\n"{}"'.format(text.description))
 
    vertices = (['({},{})'.format(vertex.x, vertex.y)
                for vertex in text.bounding_poly.vertices])
    
    ocr_text = text.description
    x1 = text.bounding_poly.vertices[0].x
    y1 = text.bounding_poly.vertices[0].y
    x2 = text.bounding_poly.vertices[1].x
    y2 = text.bounding_poly.vertices[2].y
    
    cv2.rectangle(roi_img, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
    roi_img = putText(roi_img, ocr_text, x1, y1 - 30, font_size=30)
 
if response.error.message:
    raise Exception(
        '{}\nFor more info on error messages, check: '
        'https://cloud.google.com/apis/design/errors'.format(
            response.error.message))
    
plt_imshow(["Original", "ROI"], [img, roi_img], figsize=(16, 10))


"총 내용량당 1일 영양성분 기준치에 대한 비율
나트륨 2210 mg 111%
탄수화물 2g
1%
총 내용량 1000ml
ㅇㅇㅇ 105kcal
영양정보
제품명 요리육수 한우사골
식품유형 소스, 멸균제품
내용량 1000 mL 제조원
(주)자연과사람들 / 전남 담양군
금성면 금성공단길 10
유통전문판매원
식이섬유0g
0%
풀무원식품(주)/ 충북 음성군
당류2g
2%
대소면 삼양로 730-27
지방 7g
13%
품목보고번호
20010515081456
트랜스지방0g
반품 및 교환
포화지방 3g
20%
판매원 및 구입처
콜레스테롤0mg0%
단백질 8g
15%
비타민A ORAE 0%
비타민C 0mg
0%
칼슘 45mg
6%
철분 0mg
0%
1일 영양성분 기준치에 대한 비율(%)은 2,000kcal
기준이므로 개인의 필요 열량에 따라 다를 수 있습니다.
보관법
개봉전: 실온보관(1~35°C)
개봉후: 냉장보관(0~10℃)
착률 확인해보세요
멸균팩
개봉부: LDPE
포장재질 폴리에틸렌(PE)
소비기한 상단부에 표기
보관방법 실온보관 1~35℃
원재료명 정제수, 한우진사골
농축액[한우사골(국산), 한우
잡뼈(국산)/고형분 43%],
정제소금(국산), 아미노베
이스PS[밀분해여과액(밀글
루텐:프랑스산), 효모식품
(태국산)], 복합조미식품, 대
파엑기스1, 향미증진제, 마늘추
출농축액-P, 맛있는요리애, 혼합
야채베이스
대두, 밀, 쇠고기 함유
기타 • 본 제품은 공정거래
위원회 고시 소비자
분쟁해결 기준에 의거
교환 • 보상받을 수
있습니다. • 부정
불량식품 신고는 국번
없이 1399 질소충전
▲ 주의하세요!
.
• 본 제품은 달걀, 우유, 메밀,
땅콩, 토마토, 복숭아, 호두,
잣, 닭고기, 게, 조개류(바지
대표가 같은
2111
4
12"

"총"

"내용량"

"당"

"1"

"일"

"영양"

"성분"

"기준치"

"에"

"대한"

"비율"

"나트륨"

"2210"

"mg"

"111"

"%"

"탄수화물"

"2g"

"1"

"%"

"총"

"내용

NameError: name 'plt' is not defined

In [21]:
plt_imshow(["Original", "ROI"], [img, roi_img], figsize=(16, 10))

AttributeError: module 'matplotlib' has no attribute 'figure'